[![xeus-cling](images/xeus-cling.png)](https://github.com/jupyter-xeus/xeus-cling/)

A Jupyter kernel for C++ based on the `cling` C++ interpreter and the `xeus` native implementation of the Jupyter protocol, xeus.

- GitHub repository: https://github.com/jupyter-xeus/xeus-cling/
- Online documentation: https://xeus-cling.readthedocs.io/

- C++ reference [cppreference.com ](https://en.cppreference.com/w/)

## Usage

<div style="background: #efffed;
            border: 1px solid grey;
            margin: 8px 0 8px 0;
            text-align: center;
            padding: 8px; ">
    <i class="fa-play fa" 
       style="font-size: 40px;
              line-height: 40px;
              margin: 8px;
              color: #444;">
    </i>
    <div>
    To run the selected code cell, hit <pre style="background: #efffed">Shift + Enter</pre>
    </div>
</div>

## Output and error streams

`std::cout` and `std::cerr` are redirected to the notebook frontend.

In [3]:
#include <iostream>

std::cout << "some output" << std::endl;

some output


In [4]:
std::cerr << "some error" << std::endl;

some error


In [5]:
#include <stdexcept>

In [5]:
// throw std::runtime_error("Unknown exception");

Omitting the `;` in the last statement of a cell results in an output being printed

In [51]:
int j = 5;

In [52]:
j

5

# Interpreting the C++ programming language

`cling` has a broad support of the features of C++. You can define functions, classes, templates, etc ...

## Functions

In [53]:
double sqr(double a)
{
    return a * a;
}

In [54]:
double a = 2.5;
double asqr = sqr(a);
asqr

6.2500000

## Classes

In [6]:
class Foo
{
public:

    virtual ~Foo() {}
    
    virtual void print(double value) const
    {
        std::cout << "Foo value = " << value << std::endl;
    }
};

In [7]:
Foo bar;
bar.print(1.2);

Foo value = 1.2


## Polymorphism

In [8]:
class Bar : public Foo
{
public:

    virtual ~Bar() {}
    
    virtual void print(double value) const
    {
        std::cout << "Bar value = " << 2 * value << std::endl;
    }
};

In [9]:
Foo* bar2 = new Bar;
bar2->print(1.2);
delete bar2;

Bar value = 2.4


## Templates

In [60]:
#include <typeinfo>

template <class T>
class FooT
{
public:
    
    explicit FooT(const T& t) : m_t(t) {}
    
    void print() const
    {
        std::cout << typeid(T).name() << " m_t = " << m_t << std::endl;
    }
    
private:
    
    T m_t;
};

In [61]:
FooT<double> foot(1.2);
foot.print();

input_line_83:10:52: error: use of overloaded operator '<<' is ambiguous (with operand types 'basic_ostream<char, std::char_traits<char> >' and 'const double')
        std::cout << typeid(T).name() << " m_t = " << m_t << std::endl;
        ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ^  ~~~
input_line_84:3:6: note: in instantiation of member function '__cling_N567::FooT<double>::print' requested here
foot.print();
     ^
/home/dubi/anaconda3/bin/../lib/gcc/../../x86_64-conda-linux-gnu/include/c++/9.4.0/ostream:166:7: note: candidate function
      operator<<(long __n)
      ^
/home/dubi/anaconda3/bin/../lib/gcc/../../x86_64-conda-linux-gnu/include/c++/9.4.0/ostream:170:7: note: candidate function
      operator<<(unsigned long __n)
      ^
/home/dubi/anaconda3/bin/../lib/gcc/../../x86_64-conda-linux-gnu/include/c++/9.4.0/ostream:178:7: note: candidate function
      operator<<(short __n);
      ^
/home/dubi/anaconda3/bin/../lib/gcc/../../x86_64-conda-linux-gnu/include/c++/9.4.0/ostream:1

Interpreter Error: 

## C++11 / C++14 / C++17 support

In [62]:
class Foo11
{
public:
    
    Foo11() { std::cout << "Foo11 default constructor" << std::endl; }
    Foo11(const Foo11&) { std::cout << "Foo11 copy constructor" << std::endl; }
    Foo11(Foo11&&) { std::cout << "Foo11 move constructor" << std::endl; }
};

In [63]:
Foo11 f1;
Foo11 f2(f1);
Foo11 f3(std::move(f1));

Foo11 default constructor
Foo11 copy constructor
Foo11 move constructor


In [64]:
#include <vector>

std::vector<int> v = { 1, 2, 3};
auto iter = ++v.begin();
v

{ 1, 2, 3 }

In [65]:
*iter

2

... and also lambda, universal references, `decltype`, etc ...

## Documentation and completion

 - Documentation for types of the standard library is retrieved on cppreference.com.
 - The quick-help feature can also be enabled for user-defined types and third-party libraries. More documentation on this feature is available at https://xeus-cling.readthedocs.io/en/latest/inline_help.html.


In [66]:
?std::vector

https://en.cppreference.com/w/cpp/container/vector

In [67]:
?xt::xtensor

https://xtensor.readthedocs.io/en/latest/api/xtensor.html#_CPPv2N2xt7xtensorE

## Using the `display_data` mechanism

For a user-defined type `T`, the rich rendering in the notebook and JupyterLab can be enabled by by implementing the function `nl::json mime_bundle_repr(const T& im)`, which returns the JSON mime bundle for that type.

More documentation on the rich display system of Jupyter and Xeus-cling is available at https://xeus-cling.readthedocs.io/en/latest/rich_display.html

### Image example

In [68]:
#include <string>
#include <fstream>

#include "nlohmann/json.hpp"

#include "xtl/xbase64.hpp"

namespace nl = nlohmann;

namespace im
{
    struct image
    {   
        inline image(const std::string& filename)
        {
            std::ifstream fin(filename, std::ios::binary);   
            m_buffer << fin.rdbuf();
        }
        
        std::stringstream m_buffer;
    };
    
    nl::json mime_bundle_repr(const image& i)
    {
        auto bundle = nl::json::object();
        bundle["image/png"] = xtl::base64encode(i.m_buffer.str());
        return bundle;
    }
}

In [69]:
im::image marie("images/marie.png");
marie

### Audio example

In [70]:
#include <string>
#include <fstream>

#include "nlohmann/json.hpp"

#include "xtl/xbase64.hpp"

namespace nl = nlohmann;

namespace au
{
    struct audio
    {   
        inline audio(const std::string& filename)
        {
            std::ifstream fin(filename, std::ios::binary);   
            m_buffer << fin.rdbuf();
        }
        
        std::stringstream m_buffer;
    };
    
    nl::json mime_bundle_repr(const audio& a)
    {
        auto bundle = nl::json::object();
        bundle["text/html"] =
           std::string("<audio controls=\"controls\"><source src=\"data:audio/wav;base64,")
           + xtl::base64encode(a.m_buffer.str()) +
            "\" type=\"audio/wav\" /></audio>";
        return bundle;
    }
}

In [71]:
au::audio drums("audio/audio.wav");
drums

### Display

In [72]:
#include "xcpp/xdisplay.hpp"

In [73]:
xcpp::display(drums);

### Update-display

In [74]:
#include <string>
#include "xcpp/xdisplay.hpp"

#include "nlohmann/json.hpp"

namespace nl = nlohmann;

namespace ht
{
    struct html
    {   
        inline html(const std::string& content)
        {
            m_content = content;
        }
        std::string m_content;
    };

    nl::json mime_bundle_repr(const html& a)
    {
        auto bundle = nl::json::object();
        bundle["text/html"] = a.m_content;
        return bundle;
    }
}

// A blue rectangle
ht::html rect(R"(
<div style='
    width: 90px;
    height: 50px;
    line-height: 50px;
    background-color: blue;
    color: white;
    text-align: center;'>
Original
</div>)");

In [75]:
xcpp::display(rect, "some_display_id");

Updated

In [76]:
// Update the rectangle to be red
rect.m_content = R"(
<div style='
    width: 90px;
    height: 50px;
    line-height: 50px;
    background-color: red;
    color: white;
    text-align: center;'>
Updated
</div>)";

xcpp::display(rect, "some_display_id", true);

### Clear output

In [77]:
#include <chrono>
#include <iostream>
#include <thread>

#include "xcpp/xdisplay.hpp"

In [78]:
std::cout << "hello" << std::endl;
std::this_thread::sleep_for(std::chrono::seconds(1));
xcpp::clear_output();  // will flicker when replacing "hello" with "goodbye"
std::this_thread::sleep_for(std::chrono::seconds(1));
std::cout << "goodbye" << std::endl;

goodbye


In [79]:
std::cout << "hello" << std::endl;
std::this_thread::sleep_for(std::chrono::seconds(1));
xcpp::clear_output(true);  // prevents flickering
std::this_thread::sleep_for(std::chrono::seconds(1));
std::cout << "goodbye" << std::endl;

goodbye


## Magics

Magics are special commands for the kernel that are not part of the C++ language.

They are defined with the symbol `%` for a line magic and `%%` for a cell magic.

More documentation for magics is available at https://xeus-cling.readthedocs.io/en/latest/magics.html.

In [80]:
#include <algorithm>
#include <vector>

In [81]:
std::vector<double> to_shuffle = {1, 2, 3, 4};

In [82]:
%timeit std::random_shuffle(to_shuffle.begin(), to_shuffle.end());

157 ns +- 2.46 ns per loop (mean +- std. dev. of 7 runs 10000000 loops each)


[![xtensor](images/xtensor.png)](https://github.com/xtensor-stack/xtensor/)

- GitHub repository: https://github.com/xtensor-stack/xtensor/
- Online documentation: https://xtensor.readthedocs.io/
- NumPy to xtensor cheat sheet: http://xtensor.readthedocs.io/en/latest/numpy.html

`xtensor` is a C++ library for manipulating N-D arrays with an API very similar to that of numpy.

In [84]:
#include <iostream>

#include "xtensor/xarray.hpp"
#include "xtensor/xio.hpp"
#include "xtensor/xview.hpp"

xt::xarray<double> arr1
  {{1.0, 2.0, 3.0},
   {2.0, 5.0, 7.0},
   {2.0, 5.0, 7.0}};

xt::xarray<double> arr2
  {5.0, 6.0, 7.0};

xt::view(arr1, 1) + arr2

input_line_127:2:10: fatal error: 'xtensor/xarray.hpp' file not found
#include "xtensor/xarray.hpp"
         ^~~~~~~~~~~~~~~~~~~~


Interpreter Error: 

Together with the C++ Jupyter kernel, `xtensor` offers a similar experience as `NumPy` in the Python Jupyter kernel, including broadcasting and universal functions.

In [85]:
#include <iostream>
#include "xtensor/xarray.hpp"
#include "xtensor/xio.hpp"

input_line_128:2:10: fatal error: 'xtensor/xarray.hpp' file not found
#include "xtensor/xarray.hpp"
         ^~~~~~~~~~~~~~~~~~~~


Interpreter Error: 

In [86]:
xt::xarray<int> arr
  {1, 2, 3, 4, 5, 6, 7, 8, 9};

arr.reshape({3, 3});

std::cout << arr;

input_line_129:2:2: error: use of undeclared identifier 'xt'
 xt::xarray<int> arr
 ^
input_line_129:2:16: error: expected '(' for function-style cast or type construction
 xt::xarray<int> arr
            ~~~^
input_line_129:2:18: error: use of undeclared identifier 'arr'
 xt::xarray<int> arr
                 ^
input_line_129:4:1: error: use of undeclared identifier 'arr'
arr.reshape({3, 3});
^
input_line_129:5:14: error: use of undeclared identifier 'arr'
std::cout << arr;
             ^


Interpreter Error: 

In [ ]:
#include "xtensor-blas/xlinalg.hpp"

In [ ]:
xt::xtensor<double, 2> m = {{1.5, 0.5}, {0.7, 1.0}};
std::cout << "Matrix rank: " << std::endl << xt::linalg::matrix_rank(m) << std::endl;
std::cout << "Matrix inverse: " << std::endl << xt::linalg::inv(m) << std::endl;
std::cout << "Eigen values: " << std::endl << xt::linalg::eigvals(m) << std::endl;

In [ ]:
xt::xarray<double> arg1 = xt::arange<double>(9);
xt::xarray<double> arg2 = xt::arange<double>(18);

arg1.reshape({3, 3});
arg2.reshape({2, 3, 3});

std::cout << xt::linalg::dot(arg1, arg2) << std::endl;

# CPP Object Oriented

In a **preprocessor** directives which are commands beginning with `#` are processed *before* the program is compiled 

`<<` this is known as a **stream insertion operator**
  - this points to where the data goes
  - the right to this operator is known as a **operand**

`\r` -  Carriage return. Position the screen cursor to the beginning of the current
line; do not advance to the next line

In [1]:
/* Text-printing program
      - muli line comment */

#include <iostream> // enables program to output data to the screen

//function main begins program execution
int main() {
  std::cout << "\t Welcome to C++!\r-\n"; //display message

  return 0; // incdicate that the program ended successfully 
} // end function main

main()

- Welcome to C++!


0

In [2]:
/* Get user input
   and add two integers */

// Commented out so I can run entire notebook

/* #include <iostream> // enables program to output data to the screen

//function main begins program execution
int main() {
  // DECLARING AND INITIALIZING VARIABLES
  int number1{0}; // first  integer to add      (initilized to 0)
  int number2{0}; // second integer to add      (initilized to 0)
  int sum{0};     // sum of number1 and number2 (initilized to 0)

  std::cout << "Enter first integer: "; // prompt user for data
  std::cin >> number1; // read first integer from user into number1
  std::cout << number1 << std::endl;

  std::cout << "Enter second integer: "; // prompt user for data
  std::cin >> number2; // read second integer from user into number2
  std::cout << number2 << std::endl;

  sum = number1 + number2; // add the numbers; store the result in sum

  std::cout << "Sum is " << sum << std::endl; // display sum; end line
  std::cout << "Sum is " << number1 + number2 << std::endl; //alternative

} // end function main

main() */

In [3]:
// Commented out so I don't have to worry about running entire notebook

/* Comparing integers using if statements
   and relational/equality operators */
// using std::cin;  // program uses cin
// using std::cout; // program uses cout
// using std::endl; // program uses endl

/*
using namespace std; // alternative to above

int number1{0}; // first  integer to compare  (initalized to 0)
int number2{0}; // second integer to compare  (initalized to 0)

cout << "Enter two integers to compare:\t" << endl;
cin >> number1 >> number2; // read two integers from user

if (number1 != number2)
  cout << number1 << " != " << number2 << endl;

if (number1 < number2)
  cout << number1 << " < " << number2 << endl;

if (number1 > number2)
{
  cout << number1 << " > " << number2 << endl;
}

if (number1 <= number2)
{
  cout << number1 << " <= " << number2 << endl;
}

if (number1 >= number2)
{
  cout << number1 << " >= " << number2 << endl;
}
*/

```cpp
cin >> number1 >> number2; // read two integers from user
```
Above is called **cascaded stream**

In [4]:
int main() {
    using std::cout;
    cout << "Hello World!\n";
    return 0;
}

main()

Hello World!


0

## Variable Declaration
### Compound declarations

```cpp
int x = 4;
int x {4}; //preferred
```

Its preferred because is allows narrowing conversion

```cpp
int x = 4.99; // same as int x = 4
int x {4.99}; // => error
```

### Auto Types

Types in C++ can be automatically deduced with the `auto` keyword

In [5]:
const auto x{4.99}; x

4.9900000

### Constants

**Constants data objects whose vales remain fixed during execution**

With `const` keyword variables can not be reassigned a value

```cpp
const double PI {3.14};
```

### Mixed Type Expressions

Expressions containing values of two or more data types are **allowed**

> In a mixed-type expression
> The type of each value is *promoted* to the **highest type**
>> eg `int + double => double + double` 

The data types in order from *highest to lowest* are:

`long double` | `double` | `float` |` unsigned long long int`     | `long` |
 
`long int`   | `unsigned long int` | `long int` | `unsigned int`  | `int`  |
 
`unsigned short int` | `short int` | `char` | `bool`

## Control Statements
Normally programs are executed one after the other **sequential execution**

However we can skip statements with a **transfer of control**

### Sentinel-Controlled Iteration

A **sentinel value** also called a **signal value** a **dummy value** or **flag value** 
Is used to indicate a *end of data entry*

this type of iteration can also be called a **indefinite iteration** because the number of iterations is not known before execution

### Conditional Operator

The **conditional operator** is C++'s only **ternary operator** *the operator takes three operands*

```cpp
cout << (studentGrade >= 60 ? "Passed" : "Failed");
```

### Self Explanatory 
#### Nesting Conditional Statements
#### While Loops

#### Explicit and Implicit casting

##### Explicit Casting
```cpp
double average{static_cast<double>(total) / gradeCounter};
```
If we didn't cast we could loose information from int to float conversions

> Using a cast operator in this manner is called **explicit conversion**

##### Implicit Casting

###### Promotions

**Promotions** are also known as **implicit conversion** 

#### Formatting Floating-Point Numbers
```cpp
cout << setprecision(2) << fixed; // eg 92.37
```

`setprecision` is a **parameterized stream manipulator** because it requires a argument to perform a task

to use fixed formatting instead of scientific notation use the `fixed` keyword from `<iostream>`

To use this include the header `<iomanip>` 

In contract the manipulator `endl` is known as a **non-parameterized stream manipulator**


In [6]:
#include <iostream> 
#include <iomanip> 
using namespace std;

cout << setprecision(3) << fixed << 4.332 << endl;


4.332


In [7]:
// #include <iostream>

// void ternaryOperator() {
//   using namespace std;

//   const int ROWS{10};
//   const int WIDTH{10};

//   for (int i = 1; i < ROWS; i++)
//   {
//     for (int j = 1; i < WIDTH; j++){
//       cout << (j <= i ? '*' : ' ');
//     }
    
//     cout << endl;
//   }
// }
// ternaryOperator()

In [8]:
// NESTED FOR LOOPS

#include <iostream>
#include <string>

using namespace std;

const int ROWS{10};
const int WIDTH{10};
char pattern{'A'};
int color{2};

for (int i = 0; i < 4; i++)
{
  cout << ("\x1b[3" + to_string(color++) + "m(" + pattern++ + ")");

  for (int j = 3; j < WIDTH; j++)
    cout << ' ';
  
  cout << "\t";
}
cout << endl << endl;

for (int i = 1; i < ROWS; i++)
{
  color = 2; // reset color

  cout << "\x1b[3" + to_string(color++) + "m";
  for (int j = 1; j < WIDTH; j++) // A.
    cout << (j <= i ? '*' : ' ');
  cout << "\x1b[3" + to_string(color++) + "m\t";

  for (int j = 1; j < WIDTH; j++) // B.
    cout << (WIDTH - j >= i ? '*' : ' ');
  cout << "\x1b[3" + to_string(color++) + "m\t";

  for (int j = 1; j < WIDTH; j++) // C.
    cout << (j >= i ? '*' : ' ');
  cout << "\x1b[3" + to_string(color++) + "m\t";

  for (int j = 1; j < WIDTH; j++) // D.
    cout << (WIDTH - j <= i ? '*' : ' ');
  cout << "\x1b[3" + to_string(color++) + "m\t" << endl;
}
cout << "\x1b[0m\n\n";

(A)       	(B)       	(C)       	(D)       	

*        	*********	*********	        *	
**       	******** 	 ********	       **	
***      	*******  	  *******	      ***	
****     	******   	   ******	     ****	
*****    	*****    	    *****	    *****	
******   	****     	     ****	   ******	
*******  	***      	      ***	  *******	
******** 	**       	       **	 ********	
*********	*        	        *	*********	




## Logical Operators

<div class="alert alert-danger">
Watch out for operator precedence <br>
Example <code>&&</code> takes precedence over <code>||</code>

so <code>true || true && false</code> will actually evaluate to <code>true</code>
</div>

In [9]:
bool x = true || true && false ? true : false; x // its cool that there is a warning to prevent this mistake

input_line_25:2:24: warning: '&&' within '||' [-Wlogical-op-parentheses]
 bool x = true || true && false ? true : false; x // its cool that there is a warning to prevent this mistake
               ~~ ~~~~~^~~~~~~~
input_line_25:2:24: note: place parentheses around the '&&' expression to silence this warning
 bool x = true || true && false ? true : false; x // its cool that there is a warning to prevent this mistake
                       ^
                  (            )


true

## Strings

You can represent strings with **char arrays** 
<u>or</u> we use **class string** with `#include<string>`

In [10]:
string s1{"Hi"};
string s2 = "Hello";

s1 = s2; // assign the value of s2 to s1

cout << s2 + " world \t with string length: " << to_string(s1.length());

Hello world 	 with string length: 5

In [11]:
// int day, month, year; // below is the input string to handle
// // 23/10/2021   we need to ignore all the `/` chars
// char c;

// cin >> day >> c >> month >> c >> year;
// cout << day << endl;
// cout << month << endl;
// cout << year << endl;

In [12]:
// #include <iostream>
// #include <iomanip>
// #include <string>
// using namespace std;

// cout << "Please enter your first and last name" << endl;

// string first, last;

// cin >> first >> last; // notice how the space character splits the input
// string name = first + " " + last;

// cout << "Hello, " << setw(20) << right << name << endl;

In [13]:
#include <iostream>
#include <iomanip>
#include <string>
using namespace std;

cout << setw(10) << right << "Hello, " << setw(10) << right << "World" << endl;

// setw([field legth])
// right or left kewords to align in field    right is default
// actually set w workd only once ie all subsequent values

double alpha {8.0/3.0};
double beta {9.0/3.0};

cout << alpha << endl << beta << endl; // dont know why output different to lecture
cout << showpoint << alpha << endl << beta << endl;
cout << showpoint << fixed << setprecision(4) << alpha << endl << beta << endl;
cout << showpoint << fixed << setprecision(4) << setw(10) << right << alpha << endl << beta << endl;

   Hello,      World
2.667
3.000
2.667
3.000
2.6667
3.0000
    2.6667
3.0000


## Functions

<big><strong>Function definitions are only written <u>once</u> </strong></big>

<big><strong>Function <u>cannot</u> be defined inside other functions</strong></big>

### Storage Classes

Here are some special storage-class specifiers

- auto
  - only local variables of functions can be automatic
  - automatic by default
- register
  - Suggests that variable should be placed in a high-speed register
  - Good for often-used items such as loop counters 
  - (cpp)`register int counter = 1;`
  - we usually don't use these because modern compilers can place them in high speed registers themselves
- extern 
  - Default for global variables *declared outside class or function definition*
  - Global variables are usually declared at the beginning of program
- static 
  - Variables exist for entire programs    


### Math Library Functions

We use the `<cmath>` header to get math functions such as `sqrt(900.0)`

`ceil(x)` Rounds x to the smallest integer not less than x \
`floor(x)` Rounds x to the largest integer not greater than x \
`fabs(x)` absolute value of x \
`fmod(x,y)` remainder of $x/y$ as a floating point number\
`pow(x,y)` $x$ raised to power $y$ (*x^y*)\
`sqrt(x)` square root of $x$ (*where $x$ is a nonnegative value*)\
`exp(x)` exponential function $e^x$ \
`log(x)` natural logarithm of x (*base e*)\
`log10(x)` natural logarithm of x (*base 10*)\
`sin(x)` trigonometric sine of $x$ (*x in radians*)\
`cos(x)` Trigonometric cosine of x (*x in radians*) \
`tan(x)` trigonometric tangent of $x$ (*$x$ in radians*)




In [14]:
ceil(9.2);        //==>  10.000000
ceil(-9.2);       //==> -9.0000000
floor(9.2);       //==>  9.0000000
floor(-9.2);      //==> -10.000000
fabs(-5.1);       //==>  5.1000000
fmod(2.6, 1.2);   //==>  0.20000000
pow(9,.5);        //==>  3.0000000
sqrt(9.0);        //==>  3.0000000
exp(1.0);         //==>  2.7182818
log(2.718282);    //==>  1.0000001
log10(10.0);      //==>  1.0000000
sin(0);           //==>  0.0000000
cos(0.0);         //==>  1.0000000
tan(0.0);         //==>  0.0000000

0.0000000

### Function Prototype

```cpp 
int maximum(int x, int y, int z); // function prototype
```

**Function prototype** describes a function <u>without</u> revealing its implementation

The function is described by revealing to the compiler the ...
- **function name**
- its **return type**
- types of its **parameters**

> **Note** this is different to a function declaration because it ends with a compulsory `;`

<div class="alert alert-danger">A function prototype is <b>required</b> <em>unless</em> the function is defined <u>before</u> it is used</div>

<div class="alert alert-info">It is <b>recommended</b> to always provide function prototypes <em>even if the function is used after it is already defined</em> this avoid errors & bugs <u>in the future </u> as the program evolves <hr> It is always better to eliminate errors at compile time rather than at runtime</div>

The *function name* & *parameters* is known as the **function's signature**

#### Argument Coercion

A program can call a function with a integer argument\
even though the function prototype specifies a `double` parameter\
<mark>The function will still work correctly</mark>\
provided this is not a narrowing conversion

### Standard Library Headers

`<iostream>` Contains standard input and output functions
`<iomanip>` Contains stream manipulators that format streams of data
`<cmath>` Contains math library functions
`<cstdlib>` Contains prototypes for conversions of numbers to text, text to numbers, memory allocation, random numbers and other utility functions
`<ctime>` Contains prototypes for manipulating time and data

`<arrays>` `<vector>` `<list>` `<forward_list>` `<queue>` `<dequeue>` `<stack>` `<map>` `<unordered_map>` `<set>` `<unordered_set>` `<bitset>`\
Above headers contain classes that implement standard library containers\
Containers store data during a programs execution

`<cctype>` Contains function prototypes for functions that test characters for certain properties *eg whether the character is a digit ir a punctuation* as well as functions such as upper/lowercase conversions

`<cstring>` Contains function prototypes for C-style string-processing functions

`<typeinfo>` Contains classes for runtime type identifications *determining data types ar execution time*

`exception` `stdexcept`  These headers contain classes that used for exception handling 

`<memory>` Contains classes and functions to allocate memory

`<fstream>` Contains function prototypes for functions that perform input from and output to files on a disk

`<string>` Contains the definition of class `string`

`<sstream>` Contains function prototypes for functions that perform input from and output to strings in memory

`<functional>` Contains classes and functions for algorithms

`<iterator>` Contains classes for accessing data containers

`<algorithm>` Contains functions for manipulating data in containers

`<cassert>` Contains macros for adding diagnostics that aid program debugging

`<cfloat>` Contains the floating-point size limits of the system

`<climits>` Contains the integral size limits of the system

`<cstio>` Contains function prototypes for the C-style standard input/output library functions

`<locale>` Contains classes and functions normally used by stream processing for data in the natural form for differing languages *eg, monetary formats, sorting strings, character presentation, etc*

`<limits>` Contains classes for defining the numerical data type limits on each computer platform

`<utility>` Contains classes and functions that are used by many other headers

### Random Number Generation

Use standard library `rand`

`RAND_MAX` a symbolic constant defined in the `<cstdlib>` header

In [15]:

#include <iostream> 
#include <cstdlib> //contains prototypes for functions srand and rand 
using namespace std;

auto i = rand(); // generators an unigned random integer between 0 and RAND_MAX
srand(i); // seed random number generator
srand(static_cast<unsigned int>(time(0))); // use unix time to random seed

cout << "Random integer between 0 and RAND_MAX:\t" << i << endl;
cout << "RAND_MAX:\t\t\t\t" << RAND_MAX << endl;

const int SCALING_FACTOR = 6; // this is used in what is called scaling

cout << "Rolling a six sided die:\t\t" << 1 + i % SCALING_FACTOR << endl; 
// the 1 is to remove 0 so itstead of [0,5] we have [1,6]
// think of this as a "shift factor"

// DIGIT SEPERATOR (  '  )

int num = 6'000'000; // in C++14 you could use ' as commas for big numbers
cout << "Big boy number:\t" << num << endl; 

/* Becuase these random numbers are actually psuedo-random the same sequence is used in generation to use a random seed we use the srand function from <cstdlib> */

unsigned int seed{1};
// cout << "Enter seed";
// cin >> seed;
cout << "\nWith seed:\t" << seed << endl;
srand(seed);

for (unsigned int counter{1}; counter <= 20; ++counter){
  cout << setw(10) << (1 + rand() % 6);

  if(counter % 5 == 0)
    cout << endl;
}

Random integer between 0 and RAND_MAX:	1136839577
RAND_MAX:				2147483647
Rolling a six sided die:		6
Big boy number:	6000000

With seed:	1
         2         5         4         2         6
         5         3         4         5         6
         2         2         6         4         4
         1         5         3         6         6


In [16]:
unsigned int seed{1};
// cout << "Enter seed";
// cin >> seed;
cout << "\nWith seed:\t" << seed << endl;
srand(seed);

for (unsigned int counter{1}; counter <= 20; ++counter){
  cout << setw(10) << (1 + rand() % 6);

  if(counter % 5 == 0)
    cout << endl;
}


With seed:	1
         2         5         4         2         6
         2         2         4         5         2
         3         3         6         3         5
         5         2         1         1         2


In [17]:
// time (with argument 0) from <ctime>
// returns current time as number of seconds since Jan 1, 1970 at midnight GMT
unsigned int seed{static_cast<unsigned int>(time(0))};
cout << "\nWith seed:\t" << seed << endl;
srand(seed);

for (unsigned int counter{1}; counter <= 20; ++counter){
  cout << setw(10) << (1 + rand() % 6);

  if(counter % 5 == 0)
    cout << endl;
}


With seed:	1696704442
         1         1         2         1         4
         5         6         2         4         4
         3         2         2         3         5
         3         2         1         1         5


In [18]:
// random number generation engine and distribution
#include <random> // contains C++ random number generation features

unsigned int seed{static_cast<unsigned int>(time(0))};
cout << "\nWith seed:\t" << seed << endl;

default_random_engine engine{seed};
uniform_int_distribution<unsigned int> randomInt{ 1, 6 };

for (unsigned int counter{1}; counter <= 20; ++counter){
  cout << setw(10) << randomInt(engine);

  if(counter % 5 == 0)
    cout << endl;
}


With seed:	1696704442
         1         3         4         1         2
         1         2         4         2         3
         3         5         6         4         4
         6         2         5         5         2


### Enums

> A set of integer constants represented by identifiers

<big><strong>enum</strong> <i>typeName</i> { <i>constant1</i>, <i>constant2</i>... };</big>

By default the values of these enumeration constants start at 0, unless specified otherwise and increment by 1

Cannot assign an integer to an enumeration variable

```cpp
enum Color {RED, GREEN, WHITE};

Color enumVar;
enumVar = GREEN; // cannot do enumVar = 1
```

In [19]:
enum Months { JAN = 1, FEB, MAR, APR, MAY, JUN, JUL, AUG, SEP, OCT, NOV, DEC };

Months currentMonth;
currentMonth = MAR; // cannot do currentMonth = 3
 
cout << "using Month enum " << currentMonth;

using Month enum 3

#### Scoped Enums

One problem with enum is that multiple enums may contain the same identifiers
- using enums in the same program can lead to naming collisions
- examples
```cpp
enum Status {CONTINUE, WON, LOST};
enum Looping {CONTINUE, STOP};
```
- C++ introduced **scoped enum** to solve this problem
- use scope resolution operator Status::CONTINUE

In [20]:
enum class Status {CONTINUE = 1, WON, LOST};
enum class Looping {CONTINUE, STOP};

Status gameStatus;
gameStatus = Status::CONTINUE;

if(Status::CONTINUE == gameStatus)
  cout << "Enum value from CONTINUE " << endl;

Enum value from CONTINUE 


### Better rand

> According to CERT function `rand` does not have *good statistical properties* and can be predictable which makes programs that use `rand` less secure 

C++11 provides a *more* secure library that can produce a **non-deterministic random numbers**: a set of numbers that can't be predicted

```cpp
// Using a C++11 random-number generation engine and distribution
// to roll a six-sided die

#include<iostream>
#include <random> // contains C++11 random number generation features
#include<iomanip>
#include <ctime>
using namespace std;


int main() {
  // use default random-number generation engine to
  // produce uniformly distributed pseudorandom int values from 1 to 6
  default_random_engine engine{static_cast<unsigned int>(time(0))};
  uniform_int_distribution<unsigned int> randomInt{1, 6};

  for (unsigned int counter{1}; counter <= 10; ++counter) {
    cout << setw(10) << randomInt(engine); 

    if(counter % 5 == 0) {
      cout << endl;
    } 
  }
}
```

`<unsigned int>` indicates that `uniform_int)distribution` is a **class template**\
In this case any integer type can be specified in he angle brackets

### Scopes

#### Block Scopes
This is the scope that we mean in code blocks characterized by  brackets `{}`

##### Static Variables

Local variables can be set to `static` so the *value remains after function returns to caller* so next time function is called the value remains from last completed execution 

```cpp
static unsigned int count{1};
static unsigned int count; // by default all numeric types are initialized to 0
```
#### Global scopes
An identifier declared <u>outside</u> any function or class has *global namespace scope*

#### Other Scopes
##### Function Scope
```cpp
void func(type parameter) { 
  /* Parameter with function scope here */
}
```
##### Function-Prototype

#### Local scopes



In [21]:
static unsigned int count; // by default all numeric types are initialized to 0
count

0

In [22]:
// execute this cell and bellow

int x{1}; x // global variable

1

In [23]:
void useLocal();        // function prototype
void useStaticLocal();  // function prototype
void useGlobal();       // function prototype

In [44]:

// can't declare global variables here so declare them in previous cells
// run previous cell than run this cell



int main() {
  cout << "global x in main is " << x << endl;

  int x{5}; // local variable to main
  cout << "local x in main's outer scope is " << x << endl;

  { // block starts a new scope                               
    int x{7}; // hides both x in outer scope and global x   
    cout << "local x in main's inner scope is " << x << endl;
  }

  cout << "local x in main's outer scope is " << x << endl;

  useLocal(); // useLocal has local x
  useStaticLocal(); // useStaticLocal has static local x
  useGlobal(); // useGlobal uses global x
  useLocal(); // useLocal reinitializes its local x
  useStaticLocal(); // static local x retains its prior value
  useGlobal(); // global x also retains its prior value

  cout << "\nlocal x in main is " << x << endl;
} 
main();



global x in main is 3.2000
local x in main's outer scope is 5
local x in main's inner scope is 7
local x in main's outer scope is 5

local x is 25 on entering useLocal
local x is 26 on exiting useLocal

local static x is 50 on entering useStaticLocal
local static x is 51 on exiting useStaticLocal

global x is 1 on entering useGlobal
global x is 10 on exiting useGlobal

local x is 25 on entering useLocal
local x is 26 on exiting useLocal

local static x is 51 on entering useStaticLocal
local static x is 52 on exiting useStaticLocal

global x is 10 on entering useGlobal
global x is 100 on exiting useGlobal

local x in main is 5


0

In [25]:
// useLocal reinitializes local variable x during each call
void useLocal() {
   int x{25}; // initialized each time useLocal is called

   cout << "\nlocal x is " << x << " on entering useLocal" << endl;
   ++x;
   cout << "local x is " << x << " on exiting useLocal" << endl;
}

In [26]:
// useStaticLocal initializes static local variable x only the
// first time the function is called; value of x is saved
// between calls to this function
void useStaticLocal() {
   static int x{50}; // initialized first time useStaticLocal is called

   cout << "\nlocal static x is " << x << " on entering useStaticLocal"
      << endl;
   ++x;
   cout << "local static x is " << x << " on exiting useStaticLocal"
      << endl;
}

In [27]:
// useGlobal modifies global variable x during each call
void useGlobal() {
   cout << "\nglobal x is " << x << " on entering useGlobal" << endl;
   x *= 10;
   cout << "global x is " << x << " on exiting useGlobal" << endl;
}

### Function-Call Stack and Activation Records

C++ uses a data structure called a **stack**

#### Stack Frames
As each function is called, it may call other functions each function must eventually return control to the function that had initially called it

Each time a function calls another function an entry is *pushed* onto the stack\
this entry is called a **stack frame** or an **activation record**\
it contains a **return address** that the called function needs in order to return to the calling function\
if the function returns instead of calling another function before returning\ 
the stack frame is *popped* \
and control transfers to the return address in the popped stack frame

local non-static variables need to be destroyed when the function is finished executing so non-static local variables are gone when the stack frame is popped from the function call stack

#### Stack Overflow
Self explanatory this is how [stack overflow website](stackoverflow.com) got its name

### Inline Functions
Implementing a program as a set of functions is good from a software engineering standpoint\
but function calls *involve execution-time overhead*

> C++ provides **inline functions** to help reduce function-call
overhead

Placing the qualifier `inline` before a function’s return type
in the function definition advises the compiler to generate a copy of
the function’s body code in every place where the function is called
(*when appropriate*) to avoid a function call\
**This often makes the program larger**

The compiler can ignore the `inline` qualifier and generally does so for all but the smallest functions

Reusable `inline` functions are typically placed in headers, so that their definitions can be included in each source file that uses them


In [28]:
inline double cube(const double side) {
  return side * side * side;
}

In [29]:
const int side{2};
cout << "Volume of cube with side " << 2 << "\t=\t" << cube(side) << endl;

// note becuase cube is inline cube(2) is replaced by compiler with:
// side * side * side 
// to conserve function overhead

Volume of cube with side 2	=	8.0000


@0x7f263bb3c4e0


### References and Reference Parameters

2 ways to pass arguments to functions
1. **pass-by-value**
2. **pass-by-reference**

#### Pass-by-Value
When a value is passed to function a <strong><u>copy</u></strong> of that variable is created 

> This prevents the *accidental side effects* that so greatly hinder the
development of correct and reliable software systems

<div class="alert alert-warning">One disadvantage of pass-by-value is that, if a large data item is being passed, copying that data can take a considerable amount of execution time and memory space</div>

#### Pass-by-Reference
References are aliases for other variables
    - refer to the same variable
    - can be used within a function

References must be initialized when declared
    
    cannot be reassigned as aliases to other variables


In [30]:
int count{1}; // declare variable count
int &cRef{count}; // create cRef as an alias for count
++cRef; // increment count using its alias
cout << count << endl;

/////////////////////////////

int x{3};
int&y{x}; // y refers to (is an alias for) x

cout << "x= " << x << endl << "y= " << y << endl;

y = 7;

cout << "x= " << x << endl << "y= " << y << endl;

cout << "Address of x is " << &x << endl;
cout << "Address of y is " << &y << endl;

2
x= 3
y= 3
x= 7
y= 7
Address of x is 0x7f263bce80dc
Address of y is 0x7f263bce80dc


In [31]:
void squareByValue(int number){
  number *= number;
  cout << "pass by value " << x << endl;
}
squareByValue(x);
squareByValue(x);

pass by value 7
pass by value 7


In [32]:
void squareByReference(int &number){
  number *= number;
  cout << "pass by reference " << x << endl;
}
squareByReference(x);
squareByReference(x);

pass by reference 49
pass by reference 2401



### Default Arguments

```cpp
unsigned int squareArea(unsigned int length = 1, unsigned int width = 1)
squareArea();   // A = 1
squareArea(2);  // A = 2
squareArea(3,4) // A = 12
```

### Unary Scope Resolution Operator

**Unary scope operator** `::` is used to access a global variables when a local variable of the same name is in scope

<div class="alert alert-success">We should always use unary scope operator to reference global variables because it is good programming practice</div>

In [33]:
int number{7}; // global variable named number

In [34]:
int main() {
  double number{10.5}; // local variable named number

  cout  << "Local variable number:\t"   <<   number   << endl
        << "Global variable number:\t"  << ::number   << endl;
} 

main();

Local variable number:	10.5000
Global variable number:	7


### Function Overloading

> Functions with the same name but different parameters

should perform similar tasks **look at signatures**

In [35]:
int square (double x) {
  return x * x;
}

In [36]:
int square (int x) {
  return x * x;
}

In [37]:
cout << "function overloading" << endl
     << "square(float 4.0) \t->\t" << square(4.0) << endl
     << "square(int 4) \t\t->\t"   << square(4) << endl;

function overloading
square(float 4.0) 	->	16
square(int 4) 		->	16


### Function Templates

**Compact way to make overloaded functions**

> generate separate function for different data types

Begin with keyword `template`

Formal type parameter in brackets `< >`
- every type parameter is preceded by `typename` or `class`
- specify arguments types, return types, declare variables

The function is defined as normal, except formal types used

In [38]:
template<class T> // or template<typename T> 
// ussually we use T for type but you can use whatever
T square(const T value) { // that's right we can const parameters in C++
  return value * value;
}

In [39]:
float number{10.0}; 
cout << "With user defined formal type T " << square(number);

With user defined formal type T 100.0000

## Arrays

**Arrays** are <u>fixed size</u> in of the <u>same type</u> C++\
**Vectors** on the other hand although also of the same type of sizes that are dynamically based

`<array>` and `<vector>` headers

### Declaring Arrays

Its in the form of 
```cpp
array<type, arraySize> arrayName;
```

In [40]:
// its good pragramming practice to use a const variable as the size of array
const int SIZE{10};
int myArray[SIZE];

array<int, 3> arr; // this way is called a class template array

arr[1] = 2;
arr[3] = 1;
arr[1+1] = 0;

cout << arr[1] << " " << arr[1 + 2] << endl; 

array<int, 3> arr2{2,1,0}; // alternative way to declare array
cout << arr2[1] << " " << arr2[1 + 2] << endl;

array<int, 5> n{}; // initialize elements of empty array n to 0
cout << n[1] << " " << n[1 + 2] << endl;

// Range based for loop
for (int item : arr2) {
  cout << item << " ";
}
// multiple the elements of items by 2
for (int& itemRef : arr2) {
  itemRef *= 2;
}
cout << endl;
// Display items after modifications
for (int item : arr2) {
  cout << item << " ";
}

2 1
1 0
0 0
2 1 0 
4 2 0 

If the array size is omitted, initializers determine the size

```cpp
int score[]{1, 2, 3, 4, 5}; // size is automatically initialized to 5
```

### Array Index Out of Bound
<em>C++ has <u>no</u> array bounds checking</em>

Refering to an element outside the array bound is an  <font color="red">execution time logic error</font>\
**It is not a syntax error**

## Pointers

```cpp
int count{7}; // count "directly" references a variable that contains the value 7
int countPtr{count&}; // countPtr "indirectly" references a variable that contains the value 7

int* countPtr, count;
```

### Declaration
`*` indicates the variable is a pointer\
**Note**: the whitespace around the asterisk does not matter

    typeName *variableName;

Multiple pointers require multiple asterisks

    int *myPtr1, *myPtr2;

### Null Pointer 

In previous versions either `0` or `null` as values for a **null pointer**

Since, C++11 `nullptr` is introduced as the replacement for a **null pointer**

### Pointer Operators

#### Address Operators

<big><code>&</code> address operator</big>

This returns the memory address of its operand *eg* `010FFB3C`

#### Indirection Operator

Returns the synonym for object its pointer operand points to

`*` and `&` are inverses of each other

In [41]:
double *dPtr, x, y;
dPtr = &x;
x = 3.2;
y = *dPtr * 2; y

6.4000000

In [42]:
int a;
// a is an integer
int* aPtr; // aPtr is a pointer to an integer
a = 7;
aPtr = &a; // aPtr assigned address of a

cout << "The address of a is " << &a
<< "\nThe value of aPtr is " << aPtr;
cout << "\n\nThe value of a is " << a
<< "\nThe value of *aPtr is " << *aPtr;
cout << "\n\nShowing that * and & are inverses of "
<< "each other.\n&*aPtr = " << &*aPtr
<< "\n*&aPtr = " << *&aPtr << endl;

The address of a is 0x7f263bce8178
The value of aPtr is 0x7f263bce8178

The value of a is 7
The value of *aPtr is 7

Showing that * and & are inverses of each other.
&*aPtr = 0x7f263bce8178
*&aPtr = 0x7f263bce8178


There are now three ways to pass arguments to a function
1) pass-by-value
2) pass-by-reference with a referenced argument
3) pass-by-reference with a pointer argument 🌟



In [43]:
char s[5];
cin >> setw(5) >> s;
cout << s;

char* p = s;
cin >> p;

Luca

In [51]:
#include <cstring> // strcat comes from here

char s1[20] {"Happy "};
char s2[] {"New Year "};
cout << strcat(s1, s2); // concatenate s2 to s1

// compare stings return 0 if equal
// negative if the first string is less than second string
// positive if the first string is greater than second string
cout << strcmp(s1, s2) << endl; 

const char *string1{"four"};
const char *string2{"Boston"};
cout << strlen(string1) << strlen(string2);

Happy New Year -6
46

In [57]:
 double list[ ] {1.1, 3.4, 5.5, 3.5};
 cout << &list[1] << endl;
 cout << list + 1; 

0x7f263bce8338
0x7f263bce8338

In [58]:
cout << "Hello " << endl;

          cout << "C++";

Hello 
C++

In [60]:
const int size{5};

       int array[size] {1, 2, 3, 4};

In [67]:
int x{0};
if(x == 0)
  x++;

cout << x;



1

@0x7f263bb3c4e0

In [69]:
void fun(int x, int*y, int &z) {
  z = z + 2;
  x = x * z;
  *y = z * *y;
}

int a{1}, b{2}, c{4};
fun(a, &b, c);

cout << a << endl;
cout << b << endl;
cout << c << endl;

1
12
6


```cpp
long *longPtr;      // Declare the variable longPtr to be a pointer to an object of type long
longPtr = &value1;  // Assign the address of variable value1 to longPtr
cout << *longPtr;   // Print the value of the object pointed to by longPtr
value2 = *longPtr;  // Assign the value of the object pointed to by longPtr to variable value2
cout << &value1;    // Print the address of value1
cout << longPtr;    // Print the address stored in longPtr
```